# Pre-processing

    1. Data cleaning - Replace missing values and remove duplicates
    2. Feature Engineering - Convert all data into numerical format

In [2]:
from warnings import filterwarnings

filterwarnings("ignore")

# Step 1 - Data Ingestion

Import the dataset to work upon

In [3]:
import pandas as pd

df = pd.read_csv("A:/Machine_Learning_Dev/Preprocessing/Cars93.csv", na_values=["", "NA"], keep_default_na=False)
df.head()

,id,Manufacturer,Model,Type,Min.Price,Price,Max.Price,MPG.city,MPG.highway,AirBags,...,Passengers,Length,Wheelbase,Width,Turn.circle,Rear.seat.room,Luggage.room,Weight,Origin,Make
0,1,Acura,Integra,Small,12.9,15.9,18.8,25,31,None,...,5,177,102,68,37,26.5,11.0,2705,non-USA,Acura Integra
1,2,Acura,Legend,Midsize,29.2,33.9,38.7,18,25,Driver & Passenger,...,5,195,115,71,38,30.0,15.0,3560,non-USA,Acura Legend
2,3,Audi,90,Compact,25.9,29.1,32.3,20,26,Driver only,...,5,180,102,67,37,28.0,14.0,3375,non-USA,Audi 90
3,4,Audi,100,Midsize,30.8,37.7,44.6,19,26,NaN,...,6,193,106,70,37,31.0,17.0,3405,non-USA,Audi 100
4,5,BMW,535i,Midsize,23.7,30.0,36.2,22,30,Driver only,...,4,186,109,69,39,27.0,13.0,3640,non-USA,BMW 535i


# Target feature will be weight of the vehicle 

# Step 2 - Peform basic data quality checks 

In [4]:
df.shape

(94, 28)

In [5]:
df.isnull().any()

id                    False
Manufacturer          False
Model                 False
Type                  False
Min.Price             False
Price                 False
Max.Price             False
MPG.city              False
MPG.highway           False
AirBags                True
DriveTrain            False
Cylinders             False
EngineSize            False
Horsepower            False
RPM                   False
Rev.per.mile          False
Man.trans.avail       False
Fuel.tank.capacity    False
Passengers            False
Length                False
Wheelbase             False
Width                 False
Turn.circle           False
Rear.seat.room         True
Luggage.room           True
Weight                False
Origin                False
Make                  False
dtype: bool

In [6]:
# Check for missing values and store the dataframe inside a variable
m = df.isna().sum()


In [7]:
df.isnull().sum().sum() # Shows the total number of null values in the entire dataset

np.int64(17)

In [8]:
df.isnull().any(axis=1)

0     False
1     False
2     False
3      True
4     False
      ...  
89    False
90    False
91    False
92    False
93    False
Length: 94, dtype: bool

In [9]:
m

id                     0
Manufacturer           0
Model                  0
Type                   0
Min.Price              0
Price                  0
Max.Price              0
MPG.city               0
MPG.highway            0
AirBags                4
DriveTrain             0
Cylinders              0
EngineSize             0
Horsepower             0
RPM                    0
Rev.per.mile           0
Man.trans.avail        0
Fuel.tank.capacity     0
Passengers             0
Length                 0
Wheelbase              0
Width                  0
Turn.circle            0
Rear.seat.room         2
Luggage.room          11
Weight                 0
Origin                 0
Make                   0
dtype: int64

In [10]:
m[m>0]

AirBags            4
Rear.seat.room     2
Luggage.room      11
dtype: int64

    # Taking care of missing values one by one

In [11]:
def replacer(df: pd.DataFrame, strategy='median'):
    """
    Fill missing values in dataframe.
    
    Parameters:
    - strategy: 'median', 'mean', or 'mode' for numerical columns
    """
    cat_cols = df.select_dtypes(include="object").columns.tolist()

    for col in df.columns:
        if col in cat_cols:
            mode = df[col].mode()[0] # [0] selects the top value only instead of the entire dataframe
            df[col] = df[col].fillna(mode)
        else:
            if strategy == 'median':
                fill_value = df[col].median()
            elif strategy == 'mean':
                fill_value = df[col].mean()
            else:  # mode
                fill_value = df[col].mode()[0]
            
            df[col] = df[col].fillna(fill_value)

    print(f"Missing values replaced using {strategy} for numerical columns")


"""
def replacer(df: pd.DataFrame):
    cat_cols = df.select_dtypes(include="object").columns.tolist()

    for col in df.columns:
        if col in cat_cols:
            mode = df[col].mode()[0]
            df[col] = df[col].fillna(mode)
        else:
            median = df[col].median()  # Changed from mean()
            df[col] = df[col].fillna(median)

    print("Missing values replaced in the dataframe")
"""

'\ndef replacer(df: pd.DataFrame):\n    cat_cols = df.select_dtypes(include="object").columns.tolist()\n\n    for col in df.columns:\n        if col in cat_cols:\n            mode = df[col].mode()[0]\n            df[col] = df[col].fillna(mode)\n        else:\n            median = df[col].median()  # Changed from mean()\n            df[col] = df[col].fillna(median)\n\n    print("Missing values replaced in the dataframe")\n'

1. AirBags — 4 missing (~4%)

Type: Categorical
Action: Mode Imputation

Why:

Low missing %

Categories are limited

Imputing with mode keeps distribution stable

In [12]:
# Airbags 
df['AirBags'] = df['AirBags'].fillna(df['AirBags'].mode()[0])


In [13]:
df.isnull().sum()

id                     0
Manufacturer           0
Model                  0
Type                   0
Min.Price              0
Price                  0
Max.Price              0
MPG.city               0
MPG.highway            0
AirBags                0
DriveTrain             0
Cylinders              0
EngineSize             0
Horsepower             0
RPM                    0
Rev.per.mile           0
Man.trans.avail        0
Fuel.tank.capacity     0
Passengers             0
Length                 0
Wheelbase              0
Width                  0
Turn.circle            0
Rear.seat.room         2
Luggage.room          11
Weight                 0
Origin                 0
Make                   0
dtype: int64

2. Rear.seat.room — 2 missing (~2%)

Type: Numeric (continuous measure)
Action: Median Imputation

Why:

Only 2 missing → trivial

Car measurement → could be skewed

Median is safer than mean

In [14]:
# Rear seat room 
df['Rear.seat.room'] = df['Rear.seat.room'].fillna(df['Rear.seat.room'].median())


3. Luggage.room — 11 missing (~12%)

This is the biggest problem, but still manageable.

Type: Numeric
Action: Median Imputation

Why:

12% missing is on the borderline, but still totally okay

Dropping 11 rows would reduce dataset size by 12% → not smart

Median preserves distribution without biasing extremes

In [15]:
df['Luggage.room'] = df['Luggage.room'].fillna(df['Luggage.room'].median())


In [16]:
df.shape

(94, 28)

In [17]:
df.duplicated().sum()

np.int64(1)

In [18]:
df[df.duplicated()]

,id,Manufacturer,Model,Type,Min.Price,Price,Max.Price,MPG.city,MPG.highway,AirBags,...,Passengers,Length,Wheelbase,Width,Turn.circle,Rear.seat.room,Luggage.room,Weight,Origin,Make
93,1,Acura,Integra,Small,12.9,15.9,18.8,25,31,None,...,5,177,102,68,37,26.5,11.0,2705,non-USA,Acura Integra


In [19]:
# removing the duplicate value and keeping the first 
df = df.drop_duplicates(keep="first").reset_index(drop=True)

In [20]:
df.shape

(93, 28)

In [21]:
df.nunique()

id                    93
Manufacturer          32
Model                 93
Type                   6
Min.Price             79
Price                 81
Max.Price             79
MPG.city              21
MPG.highway           22
AirBags                3
DriveTrain             3
Cylinders              6
EngineSize            26
Horsepower            57
RPM                   24
Rev.per.mile          78
Man.trans.avail        2
Fuel.tank.capacity    38
Passengers             6
Length                51
Wheelbase             27
Width                 16
Turn.circle           14
Rear.seat.room        24
Luggage.room          16
Weight                81
Origin                 2
Make                  93
dtype: int64

# Step 3 - Separate X and Y as predictor and target variable

We will predict the weight of the car

Remove the non essential columns for prediction 

We will remove the 'Weight' column as we will be testing our prediction engine to find weight by ourselves

In [22]:
x = df.drop(columns=["id", "Weight"])
x.head()

,Manufacturer,Model,Type,Min.Price,Price,Max.Price,MPG.city,MPG.highway,AirBags,DriveTrain,...,Fuel.tank.capacity,Passengers,Length,Wheelbase,Width,Turn.circle,Rear.seat.room,Luggage.room,Origin,Make
0,Acura,Integra,Small,12.9,15.9,18.8,25,31,None,Front,...,13.2,5,177,102,68,37,26.5,11.0,non-USA,Acura Integra
1,Acura,Legend,Midsize,29.2,33.9,38.7,18,25,Driver & Passenger,Front,...,18.0,5,195,115,71,38,30.0,15.0,non-USA,Acura Legend
2,Audi,90,Compact,25.9,29.1,32.3,20,26,Driver only,Front,...,16.9,5,180,102,67,37,28.0,14.0,non-USA,Audi 90
3,Audi,100,Midsize,30.8,37.7,44.6,19,26,Driver only,Front,...,21.1,6,193,106,70,37,31.0,17.0,non-USA,Audi 100
4,BMW,535i,Midsize,23.7,30.0,36.2,22,30,Driver only,Rear,...,21.1,4,186,109,69,39,27.0,13.0,non-USA,BMW 535i


In [23]:
y = df["Weight"]
y.head()

0    2705
1    3560
2    3375
3    3405
4    3640
Name: Weight, dtype: int64

# Step 4 - Data Cleaning

In [24]:
x.dtypes

Manufacturer           object
Model                  object
Type                   object
Min.Price             float64
Price                 float64
Max.Price             float64
MPG.city                int64
MPG.highway             int64
AirBags                object
DriveTrain             object
Cylinders              object
EngineSize            float64
Horsepower              int64
RPM                     int64
Rev.per.mile            int64
Man.trans.avail        object
Fuel.tank.capacity    float64
Passengers              int64
Length                  int64
Wheelbase               int64
Width                   int64
Turn.circle             int64
Rear.seat.room        float64
Luggage.room          float64
Origin                 object
Make                   object
dtype: object

In [25]:
cat_cols = x.select_dtypes(include="object").columns.tolist()

In [26]:
print(cat_cols)

['Manufacturer', 'Model', 'Type', 'AirBags', 'DriveTrain', 'Cylinders', 'Man.trans.avail', 'Origin', 'Make']


In [27]:
cont_cols = x.select_dtypes(include="number").columns.tolist() # Continuous values means numbers 

cont_cols

['Min.Price',
 'Price',
 'Max.Price',
 'MPG.city',
 'MPG.highway',
 'EngineSize',
 'Horsepower',
 'RPM',
 'Rev.per.mile',
 'Fuel.tank.capacity',
 'Passengers',
 'Length',
 'Wheelbase',
 'Width',
 'Turn.circle',
 'Rear.seat.room',
 'Luggage.room']

# Checking Cardinality

In [28]:
cat_unique = x[cat_cols].nunique() # Finding the number of unique values(cardinality) in the categorical columns - cols with text value

cat_unique

Manufacturer       32
Model              93
Type                6
AirBags             3
DriveTrain          3
Cylinders           6
Man.trans.avail     2
Origin              2
Make               93
dtype: int64

In [29]:
len(df)

93

Calculating the cardinality of each row 

In [30]:
cardinality = (cat_unique/len(df))*100
print("Values are in %")
cardinality

Values are in %


Manufacturer        34.408602
Model              100.000000
Type                 6.451613
AirBags              3.225806
DriveTrain           3.225806
Cylinders            6.451613
Man.trans.avail      2.150538
Origin               2.150538
Make               100.000000
dtype: float64

In [31]:
high_card_cols = cardinality[cardinality>90].index.tolist()
print("High cardinality columns : ",high_card_cols)

High cardinality columns :  ['Model', 'Make']


In [32]:
# Drop the high cardinality columns
x = x.drop(columns=high_card_cols)
x.head() # Checking the dataframe again to ensure the high cardinality columns are removed

,Manufacturer,Type,Min.Price,Price,Max.Price,MPG.city,MPG.highway,AirBags,DriveTrain,Cylinders,...,Man.trans.avail,Fuel.tank.capacity,Passengers,Length,Wheelbase,Width,Turn.circle,Rear.seat.room,Luggage.room,Origin
0,Acura,Small,12.9,15.9,18.8,25,31,None,Front,4,...,Yes,13.2,5,177,102,68,37,26.5,11.0,non-USA
1,Acura,Midsize,29.2,33.9,38.7,18,25,Driver & Passenger,Front,6,...,Yes,18.0,5,195,115,71,38,30.0,15.0,non-USA
2,Audi,Compact,25.9,29.1,32.3,20,26,Driver only,Front,6,...,Yes,16.9,5,180,102,67,37,28.0,14.0,non-USA
3,Audi,Midsize,30.8,37.7,44.6,19,26,Driver only,Front,6,...,Yes,21.1,6,193,106,70,37,31.0,17.0,non-USA
4,BMW,Midsize,23.7,30.0,36.2,22,30,Driver only,Rear,4,...,Yes,21.1,4,186,109,69,39,27.0,13.0,non-USA


In [33]:
# removing high cardinality columns from categorical columns 
for i in high_card_cols:
    cat_cols.remove(i)

cat_cols

['Manufacturer',
 'Type',
 'AirBags',
 'DriveTrain',
 'Cylinders',
 'Man.trans.avail',
 'Origin']

In [36]:
cont_cols

['Min.Price',
 'Price',
 'Max.Price',
 'MPG.city',
 'MPG.highway',
 'EngineSize',
 'Horsepower',
 'RPM',
 'Rev.per.mile',
 'Fuel.tank.capacity',
 'Passengers',
 'Length',
 'Wheelbase',
 'Width',
 'Turn.circle',
 'Rear.seat.room',
 'Luggage.room']

# Replacing missing values 
1. Categorical values -> Replace by Mode(most frequently occurring value)
2. Continuous values -> Replace by Mean or Median

In [51]:
def replacer(df: pd.DataFrame):
    cat_cols = df.select_dtypes(include="object").columns.tolist()
    cont_cols = df.select_dtypes(include="number").columns.tolist()

    for col in df.columns:  
        if col in cat_cols:
            mode = df[col].mode()[0]
            df[col] = df[col].fillna(mode)
        else:
            mean = df[col].mean()
            df[col] = df[col].fillna(mean)
    
    print("Missing values replaced in dataframe")


In [52]:
df.head()

,id,Manufacturer,Model,Type,Min.Price,Price,Max.Price,MPG.city,MPG.highway,AirBags,...,Passengers,Length,Wheelbase,Width,Turn.circle,Rear.seat.room,Luggage.room,Weight,Origin,Make
0,1,Acura,Integra,Small,12.9,15.9,18.8,25,31,None,...,5,177,102,68,37,26.5,11.0,2705,non-USA,Acura Integra
1,2,Acura,Legend,Midsize,29.2,33.9,38.7,18,25,Driver & Passenger,...,5,195,115,71,38,30.0,15.0,3560,non-USA,Acura Legend
2,3,Audi,90,Compact,25.9,29.1,32.3,20,26,Driver only,...,5,180,102,67,37,28.0,14.0,3375,non-USA,Audi 90
3,4,Audi,100,Midsize,30.8,37.7,44.6,19,26,Driver only,...,6,193,106,70,37,31.0,17.0,3405,non-USA,Audi 100
4,5,BMW,535i,Midsize,23.7,30.0,36.2,22,30,Driver only,...,4,186,109,69,39,27.0,13.0,3640,non-USA,BMW 535i


In [53]:
replacer(x)

Missing values replaced in dataframe


# Step 5 - Feature Engineering

1. Categorical Feature - OneHotEncoding

In [54]:
x_cat = x[cat_cols]
x_num = x[cont_cols]

In [58]:
!pip install scikit-learn

  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
   ---------------------------------------- 0.0/8.7 MB ? eta -:--:--
   --- ------------------------------------ 0.8/8.7 MB 7.5 MB/s eta 0:00:02
   --------------- ------------------------ 3.4/8.7 MB 11.0 MB/s eta 0:00:01
   ------------------------ --------------- 5.2/8.7 MB 9.5 MB/s eta 0:00:01
   --------------------------- ------------ 6.0/8.7 MB 8.0 MB/s eta 0:00:01
   ------------------------------- -------- 6.8/8.7 MB 7.2 MB/s eta 0:00:01
   ---------------------------------- ----- 7.6/8.7 MB 6.5 MB/s eta 0:00:01
   -------------------------------------- - 8.4/8.7 MB 6.0 MB/s eta 0:00:01
   ---------------------------------------- 8.7/8.7 MB 5.8 MB/s  0:00:01
   ---------------------------------------- 0.0/38.5 MB ? eta -:--:--
    --------------------------------------- 0.8/38.5 MB 7.7 MB/s eta 0:00:05
   - -------------------------------------- 1.8/38.5 MB 4.1 MB/s eta 0:00:09
   -- --------------------------


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [66]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(handle_unknown="ignore", sparse_output=False)

x_cat_pre = ohe.fit_transform(x_cat)
x_cat_pre = pd.DataFrame(x_cat_pre, columns=ohe.get_feature_names_out(x_cat.columns))



In [67]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().set_output(transform = "pandas")
x_num_pre = scaler.fit_transform(x_num)
x_num_pre.head()

,Min.Price,Price,Max.Price,MPG.city,MPG.highway,EngineSize,Horsepower,RPM,Rev.per.mile,Fuel.tank.capacity,Passengers,Length,Wheelbase,Width,Turn.circle,Rear.seat.room,Luggage.room
0,-0.485787,-0.375720,-0.282465,0.471312,0.360925,-0.841022,-0.073484,1.717489,1.129530,-1.062184,-0.083243,-0.427186,-0.286932,-0.366184,-0.610436,-0.449727,-1.037572
1,1.388017,1.497844,1.531409,-0.781032,-0.770514,0.515869,1.078322,0.369586,0.005661,0.409445,-0.083243,0.812171,1.629649,0.431983,-0.298510,0.740403,0.391972
2,1.008658,0.998227,0.948052,-0.423219,-0.581941,0.128186,0.540813,0.369586,-0.105713,0.072197,-0.083243,-0.220626,-0.286932,-0.632239,-0.610436,0.060329,0.034586
3,1.571949,1.893374,2.069191,-0.602126,-0.581941,0.128186,0.540813,0.369586,0.410659,1.359872,0.884457,0.674465,0.302785,0.165927,-0.610436,1.080441,1.106743
4,0.755752,1.091905,1.303535,-0.065407,0.172352,0.806631,1.231897,0.706562,0.430909,1.359872,-1.050944,0.192493,0.745073,-0.100128,0.013416,-0.279708,-0.322800


# Combine categorical and continuous columns

In [69]:
x_pre = x_num_pre.join(x_cat_pre)
x_pre.head()

,Min.Price,Price,Max.Price,MPG.city,MPG.highway,EngineSize,Horsepower,RPM,Rev.per.mile,Fuel.tank.capacity,...,Cylinders_3,Cylinders_4,Cylinders_5,Cylinders_6,Cylinders_8,Cylinders_rotary,Man.trans.avail_No,Man.trans.avail_Yes,Origin_USA,Origin_non-USA
0,-0.485787,-0.375720,-0.282465,0.471312,0.360925,-0.841022,-0.073484,1.717489,1.129530,-1.062184,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
1,1.388017,1.497844,1.531409,-0.781032,-0.770514,0.515869,1.078322,0.369586,0.005661,0.409445,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
2,1.008658,0.998227,0.948052,-0.423219,-0.581941,0.128186,0.540813,0.369586,-0.105713,0.072197,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
3,1.571949,1.893374,2.069191,-0.602126,-0.581941,0.128186,0.540813,0.369586,0.410659,1.359872,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
4,0.755752,1.091905,1.303535,-0.065407,0.172352,0.806631,1.231897,0.706562,0.430909,1.359872,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0


## Data Preprocessing is completed